In [1]:
from utils import get_embedding, LlamaEmbeddingFunction
import os

In [2]:
chapter_files = sorted(os.listdir('data/harry_potter'))
chapter_ids = list(map(lambda x: x.split('.')[0].replace(" ", "_"), chapter_files[:15]))

In [ ]:
# load each chapter and get the embedding
embeddings = []
for chapter_file in chapter_files[:15]:
    chapter = open(f'data/harry_potter/{chapter_file}', 'r').read()
    embedding = get_embedding(chapter)
    embeddings.append(embedding)

In [3]:
#load all chapters into memory 
chapters = []
for chapter_file in chapter_files[:15]:
    chapter = open(f'data/harry_potter/{chapter_file}', 'r').read()
    chapters.append(chapter)

In [10]:

# and get the embedding

embedding_f = LlamaEmbeddingFunction()


In [ ]:

embeddings = embedding_f(chapters)

--------

ChromaDB

In [ ]:
chapter_ids

In [6]:
import chromadb

In [7]:
client = chromadb.PersistentClient(path="chroma/hp")

In [19]:
collection = client.create_collection(name="harry2", embedding_function=embedding_f, metadata={"hnsw:space": "cosine"})
collection = client.get_collection(name="harry2", embedding_function=embedding_f)


In [13]:
metadatas = []
for id in chapter_ids:
    c_number = id.split("_")[0]
    c_name = " ".join(id.split("_")[1:])
    metadatas.append({"chapter_number": c_number, "chapter_name": c_name})

In [ ]:

collection.add(embeddings=embeddings, metadatas=metadatas, ids=chapter_ids)

In [20]:
collection.add(
    documents=chapters,
    metadatas=metadatas,
    ids=chapter_ids,
)

-------

Querying

In [22]:
question = "Who is Severus Snape and what is his position at Hogwarts?"
q_embedding = get_embedding(question)
collection.query(
    query_embeddings=q_embedding,
    n_results=5,
    include=["documents", "distances"],
    #where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"Snape"}
)

{'ids': [['012_the_mirror_of_erised',
   '014_norbert_the_norwegian_ridgeback',
   '008_the_potions_master',
   '011_quidditch_as_they_entered_november_,_the_weather_turned_very_cold',
   '007_the_sorting_hat']],
 'distances': [[0.26641811705409135,
   0.2751897531013461,
   0.27930304802405814,
   0.29993619967209295,
   0.3145420837905488]],
 'metadatas': None,
 'embeddings': None,
 'documents': [['Chapter : THE MIRROR OF ERISED .\n Christmas was coming .\n One morning in mid December , Hogwarts woke to find itself covered in several feet of snow .\n The lake froze solid and the Weasley twins were punished for bewitching several snowballs so that they followed Quirrell around , bouncing off the back of his turban .\n The few owls that managed to battle their way through the stormy sky to deliver mail had to be nursed back to health by Hagrid before they could fly off again .\n No one could wait for the holidays to start .\n While the Gryffindor common room and the Great Hall had roar

In [ ]:
chapters